I still need to use this to fix "fitmodels" so hang tight!

In [1]:
import prices

In [2]:
training=prices.pd.read_pickle("Clean Training Data.txt")
training[:10]

,LotArea,LotFrontage,SalePrice
0,8450,65.0,208500
1,9600,80.0,181500
2,11250,68.0,223500
3,9550,60.0,140000
4,14260,84.0,250000
5,14115,85.0,143000
6,10084,75.0,307000
7,10382,70.0,200000
8,6120,51.0,129900
9,7420,50.0,118000


In [3]:
# To access values of varStd, use varStd[0]

areaStd,frontStd,priceStd=prices.standardize(training.values[:,0],training.values[:,1],training.values[:,2])
designMatrix=prices.np.array([prices.np.ones(len(training)),areaStd[0],frontStd[0]]).T # ,frontStd[0]
designMatrix[:5].round(3)

array([[ 1.   , -0.286, -0.2  ],
       [ 1.   , -0.01 ,  0.622],
       [ 1.   ,  0.387, -0.036],
       [ 1.   , -0.022, -0.475],
       [ 1.   ,  1.11 ,  0.842]])

In [4]:
# All except for coefficients are assumed to be numpy arrays

# Define functions useful for algorithm steps

batch=prices.np.random.choice
def nesterov_step(labels,design,prevWeight,prevUpdate,histCoeff,learnRate):
    error = design.dot(prevWeight-histCoeff*prevUpdate)-labels
    update = (histCoeff*prevUpdate + learnRate*2*error.dot(design)).flatten()
    weights = (prevWeight - update).flatten()

    return (weights,update)

# Initialize Variables and Constants

weightsStd=prices.np.ones(prices.np.shape(designMatrix)[1])
error=designMatrix.dot(weightsStd)-priceStd[0]
lossBase=loss=prices.ssq(error)

# Try using 2e-4
learn=2e-4

# Try using 0.35
hist=0.35

iteration=1
loss_last=0
tolerance=1e-4
update=learn*2*error.dot(designMatrix)
print('Error Vector: ',error.round(3))
print('Maximum Error: ',round(max(abs(error)),2),'\n')
print('Base Loss: ',round(lossBase),'\n')
print('Initial Update: ',update.round(1))

Error Vector:  [ 0.005  1.51   0.617 ... -0.67   1.472  1.828]
Maximum Error:  11.53 

Base Loss:  5155.0 

Initial Update:  [0.6 0.7 0.7]


In [5]:
#with open("Training Progress (Copy).txt","a") as output:
while abs(loss-loss_last)>tolerance and iteration<=50:
    loss_last=loss
    miniBatch=batch(range(len(designMatrix)),300,False).sort()
    weightsStd,update=nesterov_step(priceStd[0][miniBatch],
                                 designMatrix[miniBatch],
                                 weightsStd,update,
                                 hist,learn)
    error=designMatrix.dot(weightsStd)-priceStd[0]
    loss=prices.ssq(error)
#   print('Iteration: ',iteration,'  Loss: ',loss.round(4),'\n Update: ',update,'\n')

#   string=str()
#   string=string.join(['Iteration: ',str(iteration),'  Loss: ',str(loss.round(4)),'\n Update: ',str(update),'\n\n'])
#   output.write(string)

    iteration+=1
print('DONE!','\n')
print('Final Iteration: ',iteration-1,
  '  Final Loss: ',loss.round(2),'  Base Loss: ',lossBase.round(2))


#   string=str()
#   string=string.join(['DONE!','\n\n','Final Iteration: ',str(iteration-1),
#    '  Final Loss: ',str(loss.round(2)),'  Base Loss: ',str(lossBase.round(2)),'\n\n\n\n\n'])
#   output.write(string)


DONE! 

Final Iteration:  8   Final Loss:  1190.13   Base Loss:  5154.67


In [6]:
print(weightsStd)
weights=priceStd[1]*(prices.unwrap(weightsStd,[areaStd[1],frontStd[1]],[areaStd[2],frontStd[2]]))
weights[0]+=priceStd[2]
print(weights.round(3))

[0.00040453 0.19952275 0.2559214 ]
[-2.29908783e+05  3.18800000e+00  9.33965000e+02]


In [7]:
#with open("fitmodels.py","a") as file:
#    prices.func_print(weights,'areaFront_nesterov',file)